In [1]:
import boto3
from botocore.exceptions import ClientError
import os 
os.chdir('../')

In [2]:
from utils.load_session import load_session

In [3]:
credential,session=load_session()

In [4]:
rekognition_client=session.client('rekognition')

## Some important methods in `rekognition_session`

1. `.detect_labels()`: Detect objects,scenes, and concepts in an images. 
2. `.detect_faces()`: Detect faces in an image, and return bounding boxes,facial landmarks, and other facial attributes. 
3. `.detect_text()`: Detects text in an image
4. `.detect_moderation_labels()`: Detects unsafe content in an image.  

## Common and main parameters in above methods

1.  ``Image``(**dict**): Specifies the input image is either as base64-encoded butes or as as S3 Object

        Image have two **Sub-parameters**

        `Bytes`(**bytes**): Image bytes object encoded as base 64 data. Must encode to bytes. 

        `S3Object` (dict): Image file store in S3 bucket. I takes two parameters again. a) `Bucket` (**string**), and `Name` (**String**) is `key`



2. `MaxLabels`(**int**): Maximum number of labels to return. Controls how many labels you get in the response. ***Optional for `det_labels` method***

3. `MinConfidence`(**float**): minimum confidence for consideration. 



    

Trying ``detect_labels``, and Image loading from S3 bucket.

In [5]:
## loading S3
s3_client=session.client('s3')

In [6]:
## seeing all the buckets 
bucket_names=[]
bucket_lists=s3_client.list_buckets()

for bucket in bucket_lists['Buckets']:
    bucket_names.append(bucket['Name'])


In [7]:
img_keys=[]
for content in s3_client.list_objects_v2(Bucket=bucket_names[0],Prefix='images/')['Contents']:
    img_keys.append(content['Key'])

In [8]:
img_keys

['images/cars.png', 'images/emotion_faces.jpg']

In [9]:
response_label=rekognition_client.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_names[0],
            'Name': img_keys[0]
        }
    
    },

    MinConfidence=50,
    MaxLabels=10 


)

In [10]:
response_label

{'Labels': [{'Name': 'Road',
   'Confidence': 100.0,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Transport and Logistics'}]},
  {'Name': 'Traffic Jam',
   'Confidence': 93.18080139160156,
   'Instances': [],
   'Parents': [{'Name': 'Road'},
    {'Name': 'Transportation'},
    {'Name': 'Vehicle'}],
   'Aliases': [],
   'Categories': [{'Name': 'Vehicles and Automotive'}]},
  {'Name': 'Vehicle',
   'Confidence': 93.18080139160156,
   'Instances': [],
   'Parents': [{'Name': 'Transportation'}],
   'Aliases': [],
   'Categories': [{'Name': 'Vehicles and Automotive'}]},
  {'Name': 'Outdoors',
   'Confidence': 92.67257690429688,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Nature and Outdoors'}]},
  {'Name': 'Car',
   'Confidence': 86.66761016845703,
   'Instances': [{'BoundingBox': {'Width': 0.19066843390464783,
      'Height': 0.16686566174030304,
      'Left': 2.5666826331871562e-05,
      'Top': 0.8331332206726

In [11]:
for key in response_label.keys():
    print(key)

Labels
LabelModelVersion
ResponseMetadata


Trying ``detect_labels``, and Image loading from local devices (Bytes Object)

In [12]:
from glob import glob

In [21]:

    
with open('data/images/cars.png','rb') as image_file:
    img_bytes=image_file.read()
    



In [23]:
response_label=rekognition_client.detect_labels(
    Image={
        'Bytes':img_bytes
    },
    MaxLabels=10,
    MinConfidence=75
)

In [24]:
response_label

{'Labels': [{'Name': 'Road',
   'Confidence': 100.0,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Transport and Logistics'}]},
  {'Name': 'Traffic Jam',
   'Confidence': 93.18080139160156,
   'Instances': [],
   'Parents': [{'Name': 'Road'},
    {'Name': 'Transportation'},
    {'Name': 'Vehicle'}],
   'Aliases': [],
   'Categories': [{'Name': 'Vehicles and Automotive'}]},
  {'Name': 'Transportation',
   'Confidence': 93.18080139160156,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Vehicles and Automotive'}]},
  {'Name': 'Vehicle',
   'Confidence': 93.18080139160156,
   'Instances': [],
   'Parents': [{'Name': 'Transportation'}],
   'Aliases': [],
   'Categories': [{'Name': 'Vehicles and Automotive'}]},
  {'Name': 'Outdoors',
   'Confidence': 92.67257690429688,
   'Instances': [],
   'Parents': [],
   'Aliases': [],
   'Categories': [{'Name': 'Nature and Outdoors'}]},
  {'Name': 'Car',
   'Confidence': 86.66761

## .`detect_faces()` methods

Other than common parameters it also take `Attributes` parameters

About `Attributes` parameters: 

- Type: list of strings
- Description: Specifies the facial attributes to return. The default is [``'DEFAULT'``], which returns basic attributes. [``'ALL'``] returns all available facial attributes.
- Example: ['DEFAULT'] or ['ALL']

In [27]:
response_faces=rekognition_client.detect_faces(

    Image={
        'S3Object':
        {
            'Bucket':bucket_names[0],
            'Name': img_keys[1]
        }
    },
    
    

)

In [28]:
response_faces

{'FaceDetails': [{'BoundingBox': {'Width': 0.16770029067993164,
    'Height': 0.19006715714931488,
    'Left': 0.716016411781311,
    'Top': 0.045218780636787415},
   'Landmarks': [{'Type': 'eyeLeft',
     'X': 0.763200581073761,
     'Y': 0.11629074811935425},
    {'Type': 'eyeRight', 'X': 0.8328407406806946, 'Y': 0.11593988537788391},
    {'Type': 'mouthLeft', 'X': 0.7706520557403564, 'Y': 0.1839667111635208},
    {'Type': 'mouthRight', 'X': 0.8285028338432312, 'Y': 0.18366102874279022},
    {'Type': 'nose', 'X': 0.7987505197525024, 'Y': 0.1471351534128189}],
   'Pose': {'Roll': -0.8023471236228943,
    'Yaw': -0.1536482721567154,
    'Pitch': 11.169292449951172},
   'Quality': {'Brightness': 91.77590942382812,
    'Sharpness': 46.02980041503906},
   'Confidence': 99.99625396728516},
  {'BoundingBox': {'Width': 0.1697557419538498,
    'Height': 0.17834661900997162,
    'Left': 0.4123717248439789,
    'Top': 0.04864475503563881},
   'Landmarks': [{'Type': 'eyeLeft',
     'X': 0.461790

In [31]:
response_faces['FaceDetails'][0]

{'BoundingBox': {'Width': 0.16770029067993164,
  'Height': 0.19006715714931488,
  'Left': 0.716016411781311,
  'Top': 0.045218780636787415},
 'Landmarks': [{'Type': 'eyeLeft',
   'X': 0.763200581073761,
   'Y': 0.11629074811935425},
  {'Type': 'eyeRight', 'X': 0.8328407406806946, 'Y': 0.11593988537788391},
  {'Type': 'mouthLeft', 'X': 0.7706520557403564, 'Y': 0.1839667111635208},
  {'Type': 'mouthRight', 'X': 0.8285028338432312, 'Y': 0.18366102874279022},
  {'Type': 'nose', 'X': 0.7987505197525024, 'Y': 0.1471351534128189}],
 'Pose': {'Roll': -0.8023471236228943,
  'Yaw': -0.1536482721567154,
  'Pitch': 11.169292449951172},
 'Quality': {'Brightness': 91.77590942382812, 'Sharpness': 46.02980041503906},
 'Confidence': 99.99625396728516}

In [32]:
response_faces['FaceDetails'][1]

{'BoundingBox': {'Width': 0.1697557419538498,
  'Height': 0.17834661900997162,
  'Left': 0.4123717248439789,
  'Top': 0.04864475503563881},
 'Landmarks': [{'Type': 'eyeLeft',
   'X': 0.4617904722690582,
   'Y': 0.1185457855463028},
  {'Type': 'eyeRight', 'X': 0.5325372219085693, 'Y': 0.11797602474689484},
  {'Type': 'mouthLeft', 'X': 0.46875277161598206, 'Y': 0.17956988513469696},
  {'Type': 'mouthRight', 'X': 0.5278366208076477, 'Y': 0.17905694246292114},
  {'Type': 'nose', 'X': 0.4995845854282379, 'Y': 0.15130028128623962}],
 'Pose': {'Roll': -0.8026416897773743,
  'Yaw': 1.3929450511932373,
  'Pitch': 6.916254997253418},
 'Quality': {'Brightness': 91.66661834716797, 'Sharpness': 46.02980041503906},
 'Confidence': 99.99529266357422}